<h1>Notebook for Capstone Challange</h1>

In [1]:
import pandas as pd

In [27]:
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h3>Extracting the Data</h3>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

req = requests.get(url)
tab = pd.read_html(req.text)
df_tor = pd.DataFrame(tab[0])

df_tor.columns = ['PostalCode','Borough','Neighborhood']

df_tor.reset_index(drop=True,inplace = True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h3>Creating the Data Frame</h3>

In [4]:
df_assigned = df_tor[df_tor.Borough != 'Not assigned'].reset_index(drop=True)
df_assigned.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [5]:
df_group = df_assigned.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
not_assign = df_group.Neighborhood == 'Not assigned'
df_group.loc[not_assign, 'Neighborhood'] = df_group.loc[not_assign, 'Borough']

In [7]:
df_tor_cleaned = df_group
df_tor_cleaned.shape

(103, 3)

<h3>Using the Geocoder package</h3>

In [8]:
!wget -q -O "toronto.csv" http://cocl.us/Geospatial_data 

In [9]:
geo_cor = pd.read_csv('toronto.csv')

In [10]:
geo_cor.shape
geo_cor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_tor_index = df_tor_cleaned.set_index('PostalCode')
geo_tmp = geo_cor.set_index('Postal Code')
df_toronto_comb = pd.concat([df_tor_index,geo_tmp],axis=1,join='inner')

df_toronto_comb.index.name = 'PostalCode'
df_toronto_comb.reset_index(inplace=True)

df_toronto_comb.shape

(103, 5)

In [12]:
df_toronto_comb

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


<h3>Explore and cluster the neighborhoods in Toronto</h3>

In [13]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.3.1-py_0        conda-forge
    folium:          

In [14]:
from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [15]:
df_boro = ['Central Toronto', 'East Toronto', 'West Toronto', 'Downtown Toronto']
df_central = df_toronto_comb[df_toronto_comb['Borough'].isin(df_boro)].reset_index(drop=True)
df_central.shape

(39, 5)

In [21]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

tor_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_toronto_comb['Latitude'], df_toronto_comb['Longitude'], df_toronto_comb['PostalCode'], df_toronto_comb['Borough'], df_toronto_comb['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)
    
tor_map
    

In [17]:
CLIENT_ID = 'D4CD21Y0FKZMUXOY2RIVZZ2AJVOP4SQRMONXAKYP3NYDXECX'
CLIENT_SECRET = 'SGA4JKT4FI2TJQJFKHPNEN3BQMIJQIPUXTH1TECQ0BCHXX1J'
VERSION = '20190330'

In [22]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_central['Latitude'], df_central['Longitude'], df_central['PostalCode'], df_central['Borough'], df_central['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)

(1694, 9)


In [24]:
venues_df.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()

PostalCode  Borough           Neighborhood                                                                                        
M4E         East Toronto      The Beaches                                                                                               5
M4K         East Toronto      The Danforth West,Riverdale                                                                              41
M4L         East Toronto      The Beaches West,India Bazaar                                                                            18
M4M         East Toronto      Studio District                                                                                          41
M4N         Central Toronto   Lawrence Park                                                                                             4
M4P         Central Toronto   Davisville North                                                                                          8
M4R         Central Toronto   North Toron

In [29]:
tor_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

tor_onehot['PostalCode'] = venues_df['PostalCode'] 
tor_onehot['Borough'] = venues_df['Borough'] 
tor_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fix_columns = list(tor_onehot.columns[-3:]) + list(tor_onehot.columns[:-3])
tor_onehot = tor_onehot[fix_columns]

print(tor_onehot.shape)
tor_onehot.head()

(1694, 231)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
tor_venues_freq = tor_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(tor_venues_freq.shape)

(39, 231)


In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = tor_venues_freq['PostalCode']
neighborhoods_venues_sorted['Borough'] = tor_venues_freq['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = tor_venues_freq['Neighborhoods']

for ind in np.arange(tor_venues_freq.shape[0]):
    row_categories = tor_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted




k = 4

tor_venues_freq_clustering = tor_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=k, random_state=0).fit(tor_venues_freq_clustering)

tor_clustered_df = df_central
tor_clustered_df['Cluster'] = kmeans.labels_

tor_clustered_df = tor_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
tor_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
tor_clustered_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Health Food Store,Pub,Trail,Park,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Deli / Bodega,Doner Restaurant
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
23,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,0,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Airport Terminal,Boutique,Sculpture Garden,Boat or Ferry,Plane,Bar,Harbor / Marina,Airport Gate
31,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,1,Bakery,Pharmacy,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Park,Café,Brewery,Supermarket,Bar
32,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,1,Bar,Asian Restaurant,Restaurant,Coffee Shop,Vietnamese Restaurant,Café,Pizza Place,Men's Store,Yoga Studio,Brewery
26,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,1,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Vietnamese Restaurant,Farmers Market,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Gastropub,Yoga Studio,Fish Market,Pet Store
36,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,1,Café,Coffee Shop,Pizza Place,Italian Restaurant,Sushi Restaurant,Bar,Food,Indie Movie Theater,Diner,Fish & Chips Shop
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Clothing Store,American Restaurant,Beer Bar,Italian Restaurant,Bakery,Hotel,Cocktail Bar


In [32]:
map_final = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(tor_clustered_df['Latitude'], tor_clustered_df['Longitude'], tor_clustered_df['PostalCode'], tor_clustered_df['Borough'], tor_clustered_df['Neighborhood'], tor_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_final)
       
map_final